# OpenAI Java SDK Overview

## 1. Introduction and Installation

### What is OpenAI Java SDK?

[OpenAI Java SDK](https://github.com/openai/openai-java/tree/main) is an official Java library generated by
[Stainless](https://www.stainless.com/) in pure Kotlin for interacting with the OpenAI API.
It allows developers to easily integrate language model capabilities and other OpenAI AI services into their JVM applications.


### Installation and Environment Setup

Using OpenAI Java SDK is straightforward.

For JVM projects:

```kotlin
implementation("com.openai:openai-java:$sdkVersion)
```

In Kotlin Notebook:

In [1]:
@file:DependsOn("com.openai:openai-java:4.8.0")

#### API Key

To work with the API, you need an API key that can be obtained from your personal account on the OpenAI website.

Add it to your environment variables.

[MacOS/Linux]
```bash
export OPENAI_API_KEY=<INSERT KEY HERE>
```

[Windows]
```shell
set OPENAI_API_KEY=<INSERT KEY HERE>
```

In [2]:
import com.openai.client.okhttp.OpenAIOkHttpClient

// Create a client using your API key from environment variables
// Configures using the `OPENAI_API_KEY`, `OPENAI_ORG_ID` and `OPENAI_PROJECT_ID` environment variables
val client = OpenAIOkHttpClient.fromEnv()

Let's create a simple request.

In [3]:
import com.openai.models.ChatModel
import com.openai.models.chat.completions.ChatCompletionCreateParams

val response = client.chat().completions().create(
    ChatCompletionCreateParams.builder()
        .model(ChatModel.GPT_4O_MINI)
        .addSystemMessage("You are a helpful assistant.")
        .addUserMessage("Hello! Who are you?")
        .build()
)

response.choices().first().message().content().get()

Hello! I'm an AI language model designed to assist you with a wide range of questions and tasks. Whether you need information, help with writing, or just want to chat, I'm here to help! How can I assist you today?

## 2. Basic

#### Text Requests

Basic example of interacting with the model

In [4]:
val response = client.chat().completions().create(
    ChatCompletionCreateParams.builder()
        .model(ChatModel.GPT_4O_MINI)
        .addSystemMessage("You are a Kotlin expert. You can answer questions about Kotlin.")
        .addUserMessage("Explain what coroutines are in Kotlin and provide an example.")
        .build()
)

response.choices().first().message().content().get()

Coroutines are a feature in Kotlin that allow for asynchronous programming in a more manageable and readable way compared to traditional callback-based approaches. They help in writing non-blocking code that can easily handle concurrency, making it a popular choice for applications that require tasks like network requests, I/O operations, or any long-running computations without blocking the main thread.

Kotlin coroutines are built on the concept of suspending functions, which are marked with the `suspend` keyword. These functions can be paused and resumed, allowing for cooperative multitasking within a single thread, which makes it easier to manage operations that take time to complete.

### Key Concepts:

1. **Scope**: Coroutines need to run in a coroutine scope (`CoroutineScope`), which defines the lifecycle of the coroutines launched within it.
2. **Dispatchers**: Coroutines can run on different threads using dispatchers (e.g., `Dispatchers.Main`, `Dispatchers.IO`, `Dispatchers.De

### Configuring Parameters

OpenAI SDK allows configuring various parameters that affect text generation:

In [5]:
val response = client.chat().completions().create(
    ChatCompletionCreateParams.builder()
        .model(ChatModel.GPT_4O_MINI)
        .addSystemMessage("You are a creative writer.")
        .addUserMessage("Come up with a short story about a space voyage.")
        .temperature(0.8)           // Controls randomness (0-2, default is 1)
        .maxCompletionTokens(300)   // Maximum response length
        .topP(0.95)                 // Nucleus sampling - alternative to temperature
        .presencePenalty(0.2)       // Penalty for topic repetition (-2.0 to 2.0)
        .frequencyPenalty(0.5)      // Penalty for phrase repetition (-2.0 to 2.0)
        .build()
)

response.choices().first().message().content().get()

Title: "Starlit Echoes"

The ship, *Celestial Dawn*, drifted through the velvety depths of space, its hull glinting like a jewel against the backdrop of swirling galaxies. Captain Elara Finch stood at the helm, her eyes flickering over the control panel’s myriad buttons and screens, each one alive with data and distant cosmic wonders. It had been three years since they’d embarked on their mission to explore the newly discovered exoplanet, Lyra-9, a world that promised to be rich in resources and perhaps even life.

The crew was small but dedicated—scientists, engineers, and an AI named Scribe who managed everything from navigation to morale. As they soared past nebulae bursting with colors beyond imagination, Elara often found herself lost in thought. What awaited them on Lyra-9? Would they find life? Or would it be another desolate rock adrift in the void?

One day, as the ship pierced through a dense cluster of asteroids, a sudden alarm jolted Elara from her reverie. “Captain! Uniden

### Prompt Examples

The quality of the response heavily depends on the request structure

In [6]:
// Example 1: Basic prompt
val basicPrompt = "Explain machine learning."

// Example 2: Improved prompt
val goodPrompt =
    """
    Explain the concept of machine learning to a beginner student.
    Include the following:
    1) a definition in simple words
    2) the difference between supervised and unsupervised learning
    3) three real-world application examples
    Use analogies to explain complex concepts and keep the explanation within 200 words.
    """.trimIndent()

for (prompt in listOf(basicPrompt, goodPrompt)) {
    val response = client.chat().completions().create(
        ChatCompletionCreateParams.builder()
            .model(ChatModel.GPT_4O_MINI)
            .addSystemMessage("You are a scientific expert.")
            .addUserMessage(prompt)
            .maxCompletionTokens(150)
            .build()
    )

    println("Prompt: $prompt")
    println("Response:\n${response.choices().first().message().content().get()}")
    println("-".repeat(80))
}

Prompt: Explain machine learning.
Response:
Machine learning is a subfield of artificial intelligence (AI) that focuses on the development of algorithms and statistical models that enable computers to perform specific tasks without explicit programming. Instead of following a set of pre-defined rules, machine learning algorithms learn from and make predictions or decisions based on data.

### Key Concepts in Machine Learning:

1. **Data**: Machine learning relies heavily on data, which serves as the foundation for training algorithms. Data can come in various forms, including structured data (databases, spreadsheets) and unstructured data (text, images, audio).

2. **Features**: Features are individual measurable properties or characteristics of the data being analyzed. In a dataset, features serve as input for the machine learning model.

3. **Labels
--------------------------------------------------------------------------------
Prompt: Explain the concept of machine learning to a be

## 3. Function Calling

Function calling allows models to call specific functions,
which is especially useful for extracting structured data from text or
performing specific actions based on user input.

In [7]:
import com.openai.core.JsonValue
import com.openai.models.FunctionDefinition
import com.openai.models.FunctionParameters
import com.openai.models.chat.completions.ChatCompletion
import com.openai.models.chat.completions.ChatCompletionFunctionTool
import com.openai.models.chat.completions.ChatCompletionTool

fun getWeather(location: String): Map<String, Any> = when {
    "Berlin" in location -> mapOf("temp" to 13, "conditions" to "Cloudy")
    "Tokyo" in location -> mapOf("temp" to 21, "conditions" to "Sunny")
    "New York" in location -> mapOf("temp" to 15, "conditions" to "Rainy")
    else -> mapOf("temp" to 0, "conditions" to "Unknown")
}

val functionDef = ChatCompletionFunctionTool.builder()
    .function(
        FunctionDefinition.builder()
            .name("get_weather")
            .description("Get the weather conditions in a given location")
            .parameters(
                FunctionParameters.builder()
                    .putAdditionalProperty("type", JsonValue.from("object"))
                    .putAdditionalProperty(
                        "properties", JsonValue.from(
                            mapOf(
                                "location" to JsonValue.from(
                                    mapOf(
                                        "type" to "string",
                                        "description" to "The city or location to get the weather for"
                                    )
                                )
                            )
                        )
                    )
                    .putAdditionalProperty("required", JsonValue.from(listOf("location")))
                    .putAdditionalProperty("additionalProperties", JsonValue.from(false))
                    .build()
            )
            .build()
    )
    .build()

val userQuery = "What is the weather in Berlin?"

val params = ChatCompletionCreateParams.builder()
    .model(ChatModel.GPT_4O_MINI)
    .addUserMessage(userQuery)
    .addTool(ChatCompletionTool.ofFunction(functionDef))


val response = client.chat().completions().create(params.build())

val aiMessage = response.choices().first().message()

"Response: $aiMessage"

Response: ChatCompletionMessage{content=null, refusal=null, role=assistant, annotations=[], audio=, functionCall=, toolCalls=[ChatCompletionMessageToolCall{function=ChatCompletionMessageFunctionToolCall{id=call_JaYbTlvgfeOU0EHNdTJBRG4F, function=Function{arguments={"location":"Berlin"}, name=get_weather, additionalProperties={}}, type=function, additionalProperties={}}}], additionalProperties={}}

In [8]:
import com.openai.core.jsonMapper
import com.openai.models.chat.completions.ChatCompletionToolMessageParam
import kotlin.jvm.optionals.getOrNull

if (aiMessage.toolCalls().getOrNull()?.isNotEmpty() ?: false) {
    val functionCall = aiMessage.toolCalls().get().first().asFunction()
    val functionName = functionCall.function().name()
    val functionArgs = jsonMapper().readValue(functionCall.function().arguments(), Map::class.java)

    println("The model wants to call a function: $functionName")
    println("With arguments: $functionArgs")

    if (functionName == "get_weather") {
        val location = functionArgs["location"] as? String ?: "Unknown"
        val funResponse = getWeather(location)

        val finalResponse = client.chat().completions().create(
            params
                .addMessage(aiMessage)
                .addMessage(
                    ChatCompletionToolMessageParam.builder()
                        .toolCallId(functionCall.id())
                        .content(funResponse.values.toString())
                        .build()
                )
                .build()
        )

        println("Final answer:")
        println(finalResponse.choices().first().message().content().getOrNull())
    }
}

The model wants to call a function: get_weather
With arguments: {location=Berlin}
Final answer:
The weather in Berlin is currently 13°C and cloudy.
